<a href="https://colab.research.google.com/github/AlekhyaGangopadhyay/IITPatna_Internship_LLMs/blob/main/Aria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================================
#  Aria (rhymes-ai/Aria)  ▸  Finance Harassment/Abuse Dataset
#  Full Pipeline : Train (LoRA) → Infer → Accuracy Score
#  Dataset       : iamalekhya/Finance_set  (318 videos, 9 cols)
#
#  Aria specs:
#    25.3B total params | 3.9B activated (MoE) per token
#    Multimodal-native MoE — superior video understanding
#    64K token context window
#
#  GPU Requirements:
#    bfloat16 full  → ~50 GB  → A100 80GB  (Colab Pro+)
#    8-bit quant    → ~26 GB  → A100 40GB  (Colab Pro)  ✅ recommended
#    4-bit quant    → ~14 GB  → T4 16GB    (Free Colab) ✅ fallback
# ============================================================

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 1 ── Install  (run once → Runtime ▸ Restart session)
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
!pip install "transformers>=4.48.0" accelerate sentencepiece peft bitsandbytes
!pip install rouge-score openpyxl scikit-learn opencv-python
!pip install flash-attn --no-build-isolation          # recommended for A100
!pip install grouped_gemm==0.1.6                      # optional, faster MoE routing

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 2 ── Mount Google Drive
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
from google.colab import drive
drive.mount('/content/drive')

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 3 ── ⚙️  CONFIG  ← only edit this section
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

MODEL_PATH = "rhymes-ai/Aria"

# Quantisation — choose based on your GPU:
#   "none"  → bfloat16 full precision  (A100 80GB)
#   "8bit"  → 8-bit bitsandbytes       (A100 40GB) ← recommended
#   "4bit"  → 4-bit NF4                (T4 16GB free Colab)
QUANTISATION = "4bit"

# Dataset
HF_DATASET     = "iamalekhya/Finance_set"
EXCEL_FALLBACK = "/content/drive/MyDrive/IITP_Internship/Finance_set.xlsx"

# Videos in Google Drive (all folders searched automatically)
VIDEO_DIRS = [
    "/content/drive/Shared drives/Finance_video_Anamoly/AG",
    "/content/drive/Shared drives/Finance_video_Anamoly/ag",
    "/content/drive/Shared drives/Finance_video_Anamoly/Alekhya",
    "/content/drive/Shared drives/Finance_video_Anamoly/Pritam",
    "/content/drive/Shared drives/Finance_video_Anamoly/Videos",
]
VIDEO_EXT = ".mp4"

# Training
NUM_FRAMES  = 8      # frames per video  (reduce to 4 on T4)
MAX_SEQ_LEN = 4096   # Aria supports up to 64K; 4K is safe for T4/A100
EPOCHS      = 3
BATCH_SIZE  = 1
GRAD_ACCUM  = 8
LORA_RANK   = 16
EVAL_SPLIT  = 0.10
RANDOM_SEED = 42

# Paths
OUTPUT_DIR          = "./aria_finance"
ADAPTER_DRIVE       = "/content/drive/MyDrive/IIT_Internship/aria_finance_adapter"
RESULTS_EXCEL_DRIVE = "/content/drive/MyDrive/IIT_Internship/aria_finance_results.xlsx"

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 4 ── Imports
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
import os, re, json, cv2, shutil, torch, warnings
import numpy as np
import pandas as pd
from PIL import Image
from transformers import AutoModelForCausalLM, AutoProcessor, BitsAndBytesConfig
from peft import get_peft_model, LoraConfig, TaskType
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from rouge_score import rouge_scorer as rs
from torch.utils.data import Dataset as TorchDataset
from torch.nn.utils.rnn import pad_sequence
from transformers import TrainingArguments, Trainer

warnings.filterwarnings("ignore")
print("✅ Imports OK")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 5 ── Load Dataset
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
print("Loading dataset...")
try:
    ds = load_dataset(HF_DATASET, split="train")
    df = ds.to_pandas()
    print(f"✅ Loaded from HuggingFace: {len(df)} rows")
except Exception as e:
    print(f"⚠️  HF failed ({e}), loading from Excel...")
    df = pd.read_excel(EXCEL_FALLBACK)
    print(f"✅ Loaded from Excel: {len(df)} rows")

df.columns = [c.strip() for c in df.columns]
REQUIRED = ["uni_id","Category","Aspect","Intent","Explanation","Country","Source","Sector"]
missing  = [c for c in REQUIRED if c not in df.columns]
if missing:
    raise ValueError(f"Missing columns: {missing}")

df = df.dropna(subset=REQUIRED).reset_index(drop=True)
print(f"   Rows: {len(df)} | Categories: {df['Category'].unique().tolist()}")

ALL_ASPECTS = sorted(df["Aspect"].unique().tolist())
print(f"   Aspects: {len(ALL_ASPECTS)} unique values")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 6 ── Prompt
#
#  Aria message format (official):
#    content is a LIST of dicts, each with "type" and either
#    "text" (str) or image data.
#    For images: {"type": "image"}  with None text field,
#    then pass PIL image list to processor separately.
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
def build_messages(n_frames: int, answer: str = None) -> list:
    """
    Build Aria conversation messages.
    Each frame gets its own {"text": None, "type": "image"} dict,
    followed by the text prompt.
    If answer is provided (training), includes assistant turn.
    """
    content = []

    # One image entry per frame — Aria processes each frame independently
    for i in range(n_frames):
        content.append({"text": None, "type": "image"})

    # Text instruction after all image tokens
    content.append({
        "type": "text",
        "text": f"""You are an expert video analyst specialising in financial harassment, fraud, abuse, and workplace misconduct.

The {n_frames} images above are evenly-sampled frames from a video.

Analyse the frames carefully and return a single valid JSON object with EXACTLY these fields:
{{
  "category"   : "Harassment" | "Bullying",
  "aspect"     : one of the known aspect labels listed below,
  "intent"     : 1 (intentional) | 0 (unintentional),
  "explanation": "1-2 sentence description of what is happening and why it is problematic",
  "country"    : "country name or Unknown",
  "source"     : "YouTube | News | Training | Documentary | Other",
  "sector"     : "Finance | Healthcare | Education | Retail | Tech | Government | General"
}}

Known aspect labels (pick the closest match):
{json.dumps(ALL_ASPECTS, indent=2)}

Rules:
- Output ONLY the JSON. No preamble, no markdown fences, no extra text.
- Use double quotes for all strings.
- intent must be an integer (1 or 0), not a string.

Observe carefully:
• Who is involved and what is happening
• The type and form of misconduct
• Whether the act appears deliberate
• Country and industry context clues"""
    })

    messages = [{"role": "user", "content": content}]

    if answer is not None:
        messages.append({
            "role"   : "assistant",
            "content": [{"type": "text", "text": answer}]
        })

    return messages

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 7 ── Helpers
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
def find_video(uni_id: str) -> str:
    filename = f"{uni_id}{VIDEO_EXT}"
    for folder in VIDEO_DIRS:
        path = os.path.join(folder, filename)
        if os.path.exists(path):
            return path
        if os.path.isdir(folder):
            for f in os.listdir(folder):
                if f.lower() == filename.lower():
                    return os.path.join(folder, f)
    return ""


def extract_frames(video_path: str, n: int = NUM_FRAMES) -> list:
    if not video_path or not os.path.exists(video_path):
        return []
    cap   = cv2.VideoCapture(video_path)
    total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if total == 0:
        cap.release()
        return []
    frames = []
    for idx in np.linspace(0, total - 1, n, dtype=int):
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ok, frame = cap.read()
        if ok:
            frames.append(Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)))
    cap.release()
    return frames


def row_to_json(row) -> str:
    return json.dumps({
        "category"   : str(row["Category"]).strip(),
        "aspect"     : str(row["Aspect"]).strip(),
        "intent"     : int(row["Intent"]),
        "explanation": str(row["Explanation"]).strip(),
        "country"    : str(row["Country"]).strip(),
        "source"     : str(row["Source"]).strip(),
        "sector"     : str(row["Sector"]).strip(),
    }, indent=2)


def parse_json_response(text: str) -> dict:
    try:
        m = re.search(r'\{.*\}', text, re.DOTALL)
        if m:
            return json.loads(m.group())
    except Exception:
        pass
    return {}

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 8 ── Load Model + Processor + LoRA
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
print(f"\nLoading {MODEL_PATH} [{QUANTISATION}]...")

processor = AutoProcessor.from_pretrained(
    MODEL_PATH,
    trust_remote_code = True,
)

# Build quantisation config
if QUANTISATION == "4bit":
    quant_cfg = BitsAndBytesConfig(
        load_in_4bit              = True,
        bnb_4bit_quant_type       = "nf4",
        bnb_4bit_use_double_quant = True,
        bnb_4bit_compute_dtype    = torch.bfloat16,
    )
elif QUANTISATION == "8bit":
    quant_cfg = BitsAndBytesConfig(load_in_8bit=True)
else:
    quant_cfg = None

model_kwargs = dict(
    trust_remote_code = True,
    low_cpu_mem_usage = True,
    device_map        = "auto",
)
if quant_cfg:
    model_kwargs["quantization_config"] = quant_cfg
else:
    model_kwargs["torch_dtype"] = torch.bfloat16

model = AutoModelForCausalLM.from_pretrained(MODEL_PATH, **model_kwargs)
model.eval()
print("✅ Model loaded")

# Apply LoRA to the MoE language backbone
# Aria uses Mixtral-style MoE — target both attention and MoE expert projections
lora_config = LoraConfig(
    task_type      = TaskType.CAUSAL_LM,
    r              = LORA_RANK,
    lora_alpha     = LORA_RANK * 2,
    lora_dropout   = 0.05,
    # Target attention + MoE gate/expert projections
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",   # attention
        "w1", "w2", "w3",                           # MoE expert FFN (Aria uses w1/w2/w3)
    ],
    bias = "none",
)

# Aria's language model is accessed via .language_model attribute
model.language_model = get_peft_model(model.language_model, lora_config)
model.language_model.print_trainable_parameters()
print("✅ LoRA applied to language_model (MoE backbone)")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 9 ── Build Samples
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
train_df, eval_df = train_test_split(df, test_size=EVAL_SPLIT, random_state=RANDOM_SEED)
train_df = train_df.reset_index(drop=True)
eval_df  = eval_df.reset_index(drop=True)
print(f"\nSplit → Train: {len(train_df)}  |  Eval: {len(eval_df)}")


def build_samples(dataframe, split_name="train"):
    samples, skipped = [], 0
    for i, row in dataframe.iterrows():
        uni_id     = str(row["uni_id"]).strip()
        video_path = find_video(uni_id)
        frames     = extract_frames(video_path)

        if not frames:
            print(f"  ⚠️  [{split_name}] Skipping {uni_id} — video not found")
            skipped += 1
            continue

        target = row_to_json(row)
        samples.append({
            "pil_images" : frames,
            "messages"   : build_messages(len(frames), answer=target),
            "target_json": target,
            "uni_id"     : uni_id,
        })

        if (i + 1) % 30 == 0:
            print(f"  [{split_name}] {i+1}/{len(dataframe)}  skipped={skipped}")

    print(f"  ✅ [{split_name}] ready={len(samples)}  skipped={skipped}")
    return samples


print("\n📦 Building training samples...")
train_samples = build_samples(train_df, "train")

print("\n📦 Building eval samples...")
eval_samples  = build_samples(eval_df,  "eval")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 10 ── PyTorch Dataset + Collator
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
class AriaDataset(TorchDataset):
    """
    Tokenises each sample using Aria's processor:
      processor(text=chat_text, images=pil_list, return_tensors="pt")

    pixel_values must be cast to model.dtype before forward pass.
    Labels mask the user/system portion so only the answer is trained.
    """
    def __init__(self, samples, processor, max_length=4096):
        self.samples    = samples
        self.processor  = processor
        self.max_length = max_length

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        s          = self.samples[idx]
        pil_images = s["pil_images"]
        messages   = s["messages"]   # includes assistant answer

        # apply_chat_template → full text string with special tokens
        text = self.processor.apply_chat_template(
            messages,
            add_generation_prompt = False,
        )

        # Tokenise text + images together
        inputs = self.processor(
            text         = text,
            images       = pil_images,
            return_tensors = "pt",
            max_length   = self.max_length,
            truncation   = True,
            padding      = False,
        )

        input_ids   = inputs["input_ids"].squeeze(0)
        pixel_values = inputs["pixel_values"]   # shape varies with Aria's dynamic tiling

        # Build labels — mask everything before assistant answer
        labels = input_ids.clone()

        # Find the assistant header token sequence to mask up to answer
        # Aria uses "<|im_start|>assistant" as the turn separator
        assistant_tokens = self.processor.tokenizer.encode(
            "<|im_start|>assistant\n", add_special_tokens=False
        )
        a_len     = len(assistant_tokens)
        mask_until = 0
        for pos in range(len(input_ids) - a_len):
            if input_ids[pos : pos + a_len].tolist() == assistant_tokens:
                mask_until = pos + a_len

        labels[:mask_until] = -100

        # Mask pad tokens
        pad_id = self.processor.tokenizer.pad_token_id or self.processor.tokenizer.eos_token_id
        labels[labels == pad_id] = -100

        return {
            "input_ids"     : input_ids,
            "attention_mask": torch.ones_like(input_ids),
            "labels"        : labels,
            "pixel_values"  : pixel_values,   # kept as-is; collator handles stacking
        }


def collate_fn(batch):
    pad_id = processor.tokenizer.pad_token_id or processor.tokenizer.eos_token_id

    input_ids      = pad_sequence([b["input_ids"]      for b in batch], batch_first=True, padding_value=pad_id)
    attention_mask = pad_sequence([b["attention_mask"] for b in batch], batch_first=True, padding_value=0)
    labels         = pad_sequence([b["labels"]         for b in batch], batch_first=True, padding_value=-100)

    # pixel_values: Aria uses dynamic tiling so shapes can differ between samples.
    # Concatenate along batch dim when possible; fallback to list for variable shapes.
    try:
        pixel_values = torch.cat([b["pixel_values"] for b in batch], dim=0)
    except Exception:
        pixel_values = batch[0]["pixel_values"]   # single-sample fallback

    return {
        "input_ids"     : input_ids,
        "attention_mask": attention_mask,
        "labels"        : labels,
        "pixel_values"  : pixel_values,
    }


train_torch_ds = AriaDataset(train_samples, processor, max_length=MAX_SEQ_LEN)
print(f"✅ PyTorch dataset ready: {len(train_torch_ds)} training samples")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 11 ── Train
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
model.train()
model.language_model.enable_input_require_grads()

training_args = TrainingArguments(
    output_dir                    = OUTPUT_DIR,
    per_device_train_batch_size   = BATCH_SIZE,
    gradient_accumulation_steps   = GRAD_ACCUM,
    num_train_epochs              = EPOCHS,
    learning_rate                 = 2e-4,
    bf16                          = True,
    logging_steps                 = 5,
    save_steps                    = 100,
    save_total_limit              = 2,
    warmup_ratio                  = 0.03,
    lr_scheduler_type             = "cosine",
    optim                         = "paged_adamw_8bit",
    seed                          = RANDOM_SEED,
    remove_unused_columns         = False,
    dataloader_pin_memory         = False,
    gradient_checkpointing        = True,
    gradient_checkpointing_kwargs = {"use_reentrant": False},
    report_to                     = "none",
)

# Custom Trainer to cast pixel_values to model dtype before forward pass
class AriaTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        # Aria requires pixel_values in the same dtype as model weights
        if "pixel_values" in inputs and inputs["pixel_values"] is not None:
            inputs["pixel_values"] = inputs["pixel_values"].to(
                dtype = torch.bfloat16,
                device = model.device,
            )
        return super().compute_loss(model, inputs, return_outputs=return_outputs, **kwargs)


trainer = AriaTrainer(
    model         = model,
    args          = training_args,
    train_dataset = train_torch_ds,
    data_collator = collate_fn,
)

print("\n🚀 Training started...")
trainer.train()
print("✅ Training complete!")

# Save LoRA adapter
model.language_model.save_pretrained("aria_lora_adapter")
processor.save_pretrained("aria_lora_adapter")
os.makedirs(ADAPTER_DRIVE, exist_ok=True)
shutil.copytree("aria_lora_adapter", ADAPTER_DRIVE, dirs_exist_ok=True)
print(f"✅ LoRA adapter saved → {ADAPTER_DRIVE}")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 12 ── Inference Function
#
#  Official Aria inference pattern:
#    1. apply_chat_template(messages) → text string
#    2. processor(text, images)       → input tensors
#    3. pixel_values.to(model.dtype)  → dtype match (critical!)
#    4. model.generate(**inputs, stop_strings=["<|im_end|>"])
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
model.eval()

def predict(pil_images: list) -> dict:
    """Run Aria inference on a list of PIL frames."""
    messages = build_messages(len(pil_images), answer=None)   # no answer for inference

    text = processor.apply_chat_template(
        messages,
        add_generation_prompt = True,   # True for inference only
    )

    inputs = processor(
        text   = text,
        images = pil_images,
        return_tensors = "pt",
    )

    # Cast pixel_values to model dtype — this is critical for Aria
    inputs["pixel_values"] = inputs["pixel_values"].to(
        dtype  = torch.bfloat16,
        device = model.device,
    )
    inputs = {k: v.to(model.device) if hasattr(v, "to") else v
              for k, v in inputs.items()}

    with torch.inference_mode():
        output_ids = model.generate(
            **inputs,
            max_new_tokens = 512,
            do_sample      = False,
            temperature    = 0.1,
            stop_strings   = ["<|im_end|>"],          # Aria's stop token
            tokenizer      = processor.tokenizer,     # required for stop_strings
        )

    # Decode only the newly generated tokens (exclude prompt)
    new_tokens = output_ids[0][inputs["input_ids"].shape[1]:]
    response   = processor.decode(new_tokens, skip_special_tokens=True)

    return parse_json_response(response)

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 13 ── Run Inference on ALL samples (train + eval)
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
print("\n" + "="*65)
print("  🔍 Running inference on ALL samples (train + eval)")
print("="*65)

all_samples = train_samples + eval_samples
all_results = []

for i, s in enumerate(all_samples):
    pred = predict(s["pil_images"])
    tgt  = json.loads(s["target_json"])

    all_results.append({
        "uni_id"          : s["uni_id"],
        "split"           : "train" if i < len(train_samples) else "eval",
        "true_category"   : tgt.get("category",    ""),
        "true_aspect"     : tgt.get("aspect",       ""),
        "true_intent"     : str(tgt.get("intent",   "")),
        "true_explanation": tgt.get("explanation",  ""),
        "true_country"    : tgt.get("country",      ""),
        "true_source"     : tgt.get("source",       ""),
        "true_sector"     : tgt.get("sector",       ""),
        "pred_category"   : str(pred.get("category",    "")).strip(),
        "pred_aspect"     : str(pred.get("aspect",       "")).strip(),
        "pred_intent"     : str(pred.get("intent",       "")).strip(),
        "pred_explanation": str(pred.get("explanation",  "")).strip(),
        "pred_country"    : str(pred.get("country",      "")).strip(),
        "pred_source"     : str(pred.get("source",       "")).strip(),
        "pred_sector"     : str(pred.get("sector",       "")).strip(),
    })

    if (i + 1) % 20 == 0:
        print(f"  Inferred {i+1}/{len(all_samples)}")

print(f"✅ Inference done for {len(all_results)} samples")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 14 ── Accuracy Scoring
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
scorer = rs.RougeScorer(["rougeL"], use_stemmer=True)

def score_split(rows, label="ALL"):
    if not rows:
        return {}

    def acc(t, p):
        return accuracy_score(
            [x.lower().strip() for x in t],
            [x.lower().strip() for x in p]
        )

    rouge_scores = [
        scorer.score(r["true_explanation"], r["pred_explanation"])["rougeL"].fmeasure
        for r in rows
    ]
    avg_rouge = float(np.mean(rouge_scores))

    cat_acc = acc([r["true_category"] for r in rows], [r["pred_category"] for r in rows])
    asp_acc = acc([r["true_aspect"]   for r in rows], [r["pred_aspect"]   for r in rows])
    int_acc = acc([r["true_intent"]   for r in rows], [r["pred_intent"]   for r in rows])
    cty_acc = acc([r["true_country"]  for r in rows], [r["pred_country"]  for r in rows])
    sec_acc = acc([r["true_sector"]   for r in rows], [r["pred_sector"]   for r in rows])
    src_acc = acc([r["true_source"]   for r in rows], [r["pred_source"]   for r in rows])

    overall = (cat_acc*0.25 + asp_acc*0.25 + int_acc*0.15 +
               avg_rouge*0.20 + sec_acc*0.10 + cty_acc*0.03 + src_acc*0.02)

    return {
        "split"             : label,
        "n_samples"         : len(rows),
        "category_acc"      : round(cat_acc   * 100, 2),
        "aspect_acc"        : round(asp_acc   * 100, 2),
        "intent_acc"        : round(int_acc   * 100, 2),
        "country_acc"       : round(cty_acc   * 100, 2),
        "source_acc"        : round(src_acc   * 100, 2),
        "sector_acc"        : round(sec_acc   * 100, 2),
        "explanation_rougeL": round(avg_rouge * 100, 2),
        "OVERALL_SCORE_%"   : round(overall   * 100, 2),
    }


train_rows   = [r for r in all_results if r["split"] == "train"]
eval_rows    = [r for r in all_results if r["split"] == "eval"]
train_scores = score_split(train_rows,  "TRAIN")
eval_scores  = score_split(eval_rows,   "EVAL")
all_scores   = score_split(all_results, "ALL")

def print_scores(s: dict):
    print(f"\n  ── {s['split']} SET  ({s['n_samples']} samples) ──")
    print(f"   Category    Accuracy  : {s['category_acc']:>7.2f} %   (weight 25%)")
    print(f"   Aspect      Accuracy  : {s['aspect_acc']:>7.2f} %   (weight 25%)")
    print(f"   Intent      Accuracy  : {s['intent_acc']:>7.2f} %   (weight 15%)")
    print(f"   Explanation ROUGE-L   : {s['explanation_rougeL']:>7.2f} %   (weight 20%)")
    print(f"   Sector      Accuracy  : {s['sector_acc']:>7.2f} %   (weight 10%)")
    print(f"   Country     Accuracy  : {s['country_acc']:>7.2f} %   (weight  3%)")
    print(f"   Source      Accuracy  : {s['source_acc']:>7.2f} %   (weight  2%)")
    print(f"   {'─'*42}")
    print(f"   ⭐ OVERALL SCORE      : {s['OVERALL_SCORE_%']:>7.2f} %")

print("\n" + "="*65)
print("              📊 ACCURACY REPORT")
print("="*65)
print_scores(train_scores)
print_scores(eval_scores)
print_scores(all_scores)
print("="*65)

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 15 ── Save Results to Excel (7 sheets)
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
results_df = pd.DataFrame(all_results)
summary_df = pd.DataFrame([train_scores, eval_scores, all_scores])

os.makedirs(os.path.dirname(RESULTS_EXCEL_DRIVE), exist_ok=True)

with pd.ExcelWriter(RESULTS_EXCEL_DRIVE, engine="openpyxl") as writer:
    summary_df.to_excel(writer, sheet_name="Accuracy Summary", index=False)
    results_df.to_excel(writer, sheet_name="Per Sample",       index=False)

    for field in ["category", "aspect", "intent", "sector", "country", "source"]:
        breakdown = results_df[[
            "uni_id", "split",
            f"true_{field}", f"pred_{field}"
        ]].copy()
        breakdown["correct"] = (
            breakdown[f"true_{field}"].str.lower().str.strip() ==
            breakdown[f"pred_{field}"].str.lower().str.strip()
        )
        breakdown.to_excel(writer, sheet_name=f"{field.capitalize()} Detail", index=False)

print(f"\n✅ Results saved → {RESULTS_EXCEL_DRIVE}")
print("\n🎉 ALL DONE!")